### 数据集
- 我们需要以理想的格式生成数据集，以便于两个预训练任务：遮蔽语言模型和下一句预测。
- 一方面，最初的BERT模型是在两个庞大的图书语料库和英语维基百科的合集上预训练的，但它很难吸引这本书的大多数读者。
- 另一方面，现成的预训练BERT模型可能不适合医学等特定领域的应用。因此，在定制的数据集上对BERT进行预训练变得越来越流行。
- 为了方便BERT预训练的演示，我们使用了较小的语料库WikiText-2。
- 与用于预训练word2vec的PTB数据集相比，WikiText-2（1）保留了原来的标点符号，适合于下一句预测；（2）保留了原来的大小写和数字；（3）大了一倍以上。

In [467]:
import os
import random
import torch
from d2l import torch as d2l

- 在WikiText-2数据集中，每行代表一个段落，其中在任意标点符号及其前面的词元之间插入空格。
- 保留至少有两句话的段落。为了简单起见，我们仅使用句号作为分隔符来拆分句子。

In [468]:
d2l.DATA_HUB['wikitext-2'] = (
    'https://s3.amazonaws.com/research.metamind.io/wikitext/'
    'wikitext-2-v1.zip', '3c914d17d80b1459be871a5039ac23e752a53cbe')

def _read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    # 大写字母转换为小写字母
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs

可以看出文本的加载形式，文本被加载成列表的形式，每个列表又是一个储存了多个句子的列表.
里面的为止部分已经替换成了unk

In [469]:
data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
paragraphs = _read_wiki(data_dir)
paragraphs[2]

['major changes to the routing of m @-@ 47 started in december 1960 when the i @-@ 75 / us 10 / us 23 freeway opened between saginaw and bay city',
 'us 10 was rerouted east of midland to bay city along the m @-@ 20 freeway',
 'm @-@ 47 was rerouted along the former us 10 from saginaw to east of midland using a connector expressway from freeland north to the us 10 freeway',
 'm @-@ 81 was extended over state street in saginaw , and the former routing of m @-@ 47 between saginaw and bay city was redesignated as m @-@ 84',
 'm @-@ 13 also replaced the former us 23 / m @-@ 47 when us 23 was moved to freeways as well',
 'the southern end of m @-@ 47 was changed in 1962 with the completion of i @-@ 96 in the lansing area',
 'us 16 was replaced by m @-@ 43 , and the southern terminus of m @-@ 47 was moved to exit 122 along i @-@ 96',
 'this segment of m @-@ 47 south of m @-@ 46 became an extension of m @-@ 52 in 1969 , <unk> m @-@ 47 to hemlock',
 'the interchange at <unk> road north of free

接下来是bert所需的输入数据：
- bert和以往模型不一样的地方在于，有非常多的输入数据
- 1、和其他模型一样，语料库相关数据，
- 2、segment，对于句子对进行前后句子区分
- 3、句子有效长度
- 4、掩码矩阵权重、掩码信息
- 5、下一个句子预测训练样本

先为BERT的两个预训练任务实现辅助函数。这些辅助函数将在稍后将原始文本语料库转换为理想格式的数据集时调用，以预训练BERT。

_get_next_sentence函数生成二分类任务的训练样本。

In [470]:
def _get_next_sentence(sentence, next_sentence, paragraphs):
    """这个函数的目的就是在于生成二分类的任务
    输入一个句子sentence,以及其真实的下一个句子
    有一半概率, 我接下来的句子就是真实句子
    但有有一半概率我把这个真实的next_sentence丢掉, 
    直接从paragraphs里面选择一个句子来作为next_sentence
    那么我就构建好了这么一个预测模型:
        如果next_sentence确实是,那么is_next就是True
        反之is_next就是False, 所以is_next就是标签
    """
    if random.random() < 0.5:
        is_next = True
    else:
        # paragraphs是三重列表的嵌套
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next

- 下面的函数通过调用_get_next_sentence函数从输入paragraph生成用于下一句预测的训练样本。
- 这里paragraph是句子列表，其中每个句子都是词元列表。
- 自变量max_len指定预训练期间的BERT输入序列的最大长度。

In [471]:
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        # 这里把每个句子
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        # 考虑1个'<cls>'词元和2个'<sep>'词元
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        # 这里生成的tokens, segments就可以作为bert的输入了
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        # 这里是构建下一个句子预测的样本,这里可以看到放入的是tokens, segments,而不是token_a, token_b
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

演示一下如何使用

In [472]:
paragraphs = [d2l.tokenize(
    paragraph, token='word') for paragraph in paragraphs]

此时paragraphs是将整个文章彻底拆开

In [473]:
paragraphs[2][0]

['major',
 'changes',
 'to',
 'the',
 'routing',
 'of',
 'm',
 '@-@',
 '47',
 'started',
 'in',
 'december',
 '1960',
 'when',
 'the',
 'i',
 '@-@',
 '75',
 '/',
 'us',
 '10',
 '/',
 'us',
 '23',
 'freeway',
 'opened',
 'between',
 'saginaw',
 'and',
 'bay',
 'city']

In [474]:

sentences = [sentence for paragraph in paragraphs
                for sentence in paragraph]

sentences是把所有句子全部拉成一个列表，不再区分paragraph

In [475]:
sentences[12]

['us',
 '10',
 'was',
 'rerouted',
 'east',
 'of',
 'midland',
 'to',
 'bay',
 'city',
 'along',
 'the',
 'm',
 '@-@',
 '20',
 'freeway']

In [476]:
vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>', '<mask>', '<cls>', '<sep>'])

In [477]:
for paragraph in paragraphs:
    sample = _get_nsp_data_from_paragraph(paragraph,paragraphs,vocab,64)
    break

可以看到，_get_nsp_data_from_paragraph将句子处理成了[[sentence]，[segment]]形式

In [478]:
sample[0][0][:10]

['<cls>',
 'by',
 'the',
 '1890s',
 ',',
 'the',
 'observatory',
 'had',
 'become',
 'crowded']

In [479]:
sample[0][1][30:]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

### 生成遮蔽语言模型任务的数据
- 为了从BERT输入序列生成遮蔽语言模型的训练样本，我们定义了以下_replace_mlm_tokens函数。
- 在其输入中，tokens是表示BERT输入序列的词元的列表，candidate_pred_positions是不包括特殊词元的BERT输入序列的词元索引的列表（特殊词元在遮蔽语言模型任务中不被预测），以及num_mlm_preds指示预测的数量（选择15%要预测的随机词元）。在 14.8.5.1节中
- 定义遮蔽语言模型任务之后，在每个预测位置，输入可以由特殊的“掩码”词元或随机词元替换，或者保持不变。最后，该函数返回可能替换后的输入词元、发生预测的词元索引和这些预测的标签。

In [480]:
"""
BERT预训练任务中被选中的15%的词中仅80%的词是真正被Masked掉的,另外10%不变，最后10%随机换为其他词，
"""
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds,
                        vocab):
    # 为遮蔽语言模型的输入创建新的词元副本，其中输入可能包含替换的“<mask>”或随机词元
    """
    后面代码显示 num_mlm_preds = max(1, round(len(tokens) * 0.15)) 
    num_mlm_preds为整体token长度的15%，至少为1
    
    candidate_pred_positions是把tokens里面不是'<cls>', '<sep>'的词元索引拿出来
    
    """
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    # 打乱后用于在遮蔽语言模型任务中获取15%的随机词元进行预测
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:
        # num_mlm_preds用于控制百分之多少的词用于替换
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        ##################################################
        # 下面这部分是按比例去生成masked_token
        # 80%的时间：将词替换为“<mask>”词元
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            # 10%的时间：保持词不变
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            # 10%的时间：用随机词替换该词
            else:
                masked_token = random.choice(vocab.idx_to_token)
        #################################################    
        #  mlm_pred_position是candidate_pred_positions中选出是索引
        mlm_input_tokens[mlm_pred_position] = masked_token
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    # mlm_input_tokens：token替换后的token_list, pred_positions_and_labels :待预测索引和标签
    return mlm_input_tokens, pred_positions_and_labels

In [481]:
batch_size, max_len = 512, 64
for tokens, segments, is_next in _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    print(tokens,end='\n')
    print(segments,end='\n')
    print(is_next,end='\n')
    break

['<cls>', 'by', 'the', '1890s', ',', 'the', 'observatory', 'had', 'become', 'crowded', 'by', 'the', 'rapidly', 'growing', 'university', '<sep>', '"', 'in', '2014', ',', 'jack', '<unk>', 'of', 'what', 'culture', 'ranked', 'him', 'as', 'the', 'second', 'greatest', 'ever', 'fighting', 'game', 'character', '.', '<sep>']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
False


In [482]:
candidate_pred_positions = []
# tokens是一个字符串列表
for i, token in enumerate(tokens):
    # 在遮蔽语言模型任务中不会预测特殊词元
    if token in ['<cls>', '<sep>']:
        continue
    candidate_pred_positions.append(i)
num_mlm_preds = max(1, round(len(tokens) * 0.15))

- 函数演示,和上面原始的tokens对比，可以看到mlm_input_tokens已经把部分词替换成了mask，部分词进行了随机替换
- pred_positions_and_labels储存的是你具体吧哪个位置进行了替换，替换前是啥样
  - 比如(14, 'under')就是把第14个的'under'替换成了mask

In [483]:
mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(tokens,candidate_pred_positions,num_mlm_preds,vocab)
print(mlm_input_tokens)
print(pred_positions_and_labels)

['<cls>', 'by', 'the', '1890s', '<mask>', 'the', 'observatory', 'had', 'become', 'crowded', 'by', 'the', 'rapidly', 'growing', 'university', '<sep>', '"', 'in', '2014', ',', 'jack', '<unk>', 'of', 'what', 'culture', 'ranked', '<mask>', 'as', 'the', '<mask>', 'greatest', 'ever', '<mask>', 'game', 'knee', '<mask>', '<sep>']
[(35, '.'), (34, 'character'), (29, 'second'), (32, 'fighting'), (26, 'him'), (4, ',')]


通过调用前述的_replace_mlm_tokens函数，以下函数将BERT输入序列（tokens）作为输入，并返回输入词元的索引、发生预测的词元索引以及这些预测的标签索引。

In [484]:
def _get_mlm_data_from_tokens(tokens, vocab):
    candidate_pred_positions = []
    # tokens是一个字符串列表
    for i, token in enumerate(tokens):
        # 在遮蔽语言模型任务中不会预测特殊词元
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)
    # 遮蔽语言模型任务中预测15%的随机词元
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(
        tokens, candidate_pred_positions, num_mlm_preds, vocab)
    pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    # 输入词元的索引、发生预测的词元索引以及这些预测的标签索引。
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

依旧是演示
- vocab[mlm_input_tokens]返回的是vocab里面每个token的索引
- pred_positions就是替换的位置
- vocab[mlm_pred_labels]是具体替换的词在vocab的索引

In [485]:
pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
pred_positions_and_labels

[(4, ','),
 (26, 'him'),
 (29, 'second'),
 (32, 'fighting'),
 (34, 'character'),
 (35, '.')]

### 将文本转换为预训练数据集

BERT预训练定制一个Dataset类。在此之前，我们仍然需要定义辅助函数_pad_bert_inputs来将特殊的“<mask>”词元附加到输入。它的参数examples包含来自两个预训练任务的辅助函数_get_nsp_data_from_paragraph和_get_mlm_data_from_tokens的输出。

In [486]:
def _pad_bert_inputs(examples, max_len, vocab):
    # max_len是用于控制单个sentence的长度的
    max_num_mlm_preds = round(max_len * 0.15)
    all_token_ids, all_segments, valid_lens,  = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    # 这一段很明显是要吧单个example里面的全部数据进行一次整合，一个example对应一个paragraph的话，要整合到sentence的级别
    for (token_ids, pred_positions, mlm_pred_label_ids, segments,
         is_next) in examples:
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (
            max_len - len(token_ids)), dtype=torch.long))
        all_segments.append(torch.tensor(segments + [0] * (
            max_len - len(segments)), dtype=torch.long))
        # valid_lens不包括'<pad>'的计数
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (
            max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # 填充词元的预测将通过乘以0权重在损失中过滤掉
        all_mlm_weights.append(
            torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (
                max_num_mlm_preds - len(pred_positions)),
                dtype=torch.float32))
        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (
            max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    return (all_token_ids, all_segments, valid_lens, all_pred_positions,
            all_mlm_weights, all_mlm_labels, nsp_labels)

先看看example长啥样

In [487]:
examples = []
for paragraph in paragraphs:
    examples.extend(_get_nsp_data_from_paragraph(
        paragraph, paragraphs, vocab, max_len))
# 获取遮蔽语言模型任务的数据
examples = [(_get_mlm_data_from_tokens(tokens, vocab)
                + (segments, is_next))
                for tokens, segments, is_next in examples]

继续演示

In [488]:
max_num_mlm_preds = round(max_len * 0.15)
all_token_ids, all_segments, valid_lens,  = [], [], []
all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
nsp_labels = []
for (token_ids, pred_positions, mlm_pred_label_ids, segments,
        is_next) in examples:
    # 长度不够的话，用'<pad>'进行填充，当然这里只能返回填充的索引
    all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (max_len - len(token_ids)), dtype=torch.long))
    # segments，长度不够填充为0
    all_segments.append(torch.tensor(segments + [0] * (max_len - len(segments)), dtype=torch.long))
    # 句子有效长度统计
    valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
    # mask替换位置，因为规定是替换15%的token，但是具体到每个sentence，长度会不够，同样用0 补齐
    all_pred_positions.append(torch.tensor(pred_positions + [0] * (max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
    # 填充词元的预测将通过乘以0权重在损失中过滤掉
    all_mlm_weights.append(torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (max_num_mlm_preds - len(pred_positions)),dtype=torch.float32))
    
    all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
    
    nsp_labels.append(torch.tensor(is_next, dtype=torch.long))

all_token_ids储存了sentence里面的句子在vocab的索引，$'<pad>'$索引为1

In [499]:
all_token_ids[0]

tensor([   3,   22,    2, 6320, 9883,    5, 3243,    2,  354, 8634,   22,    5,
        1894, 1683,  311,    4,   37,    2,   13,  166,    2,    2,   35,  196,
          10, 2494,   59, 1655, 2535,    6,    8,  666,   26,   13,  336,   10,
          35,    5,   40,  731,  688,   17, 2494,   33,   32, 2535,    4,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1])

In [500]:
all_segments[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [501]:
valid_lens[0]

tensor(47.)

In [502]:
all_pred_positions[0]

tensor([ 2,  4,  6,  7, 17, 20, 21,  0,  0,  0])

In [503]:
all_mlm_weights[0]

tensor([1., 1., 1., 1., 1., 1., 1., 0., 0., 0.])

In [504]:
all_mlm_labels[0]

tensor([   5,    6, 3243,   33, 1210,  454,   10,    0,    0,    0])

In [505]:
nsp_labels[0]

tensor(0)

看到这里，我们总算可以对目前的处理进行总结，李沐大佬把目前的所有处理数据整合成了一个example，方便后续进行计算，一个example对应一个paragraph

可以看出一个标准example是包含了_get_nsp_data_from_paragraph和_get_mlm_data_from_tokens的所有东西，包括如下东西
- _get_nsp_data_from_paragraph的返回：tokens, segments, is_next
- _get_mlm_data_from_tokens 的返回：vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]
- 也就是说包含如下信息
  - tokens:句子对
    - 注意下，本代码的定义中：tokens = ['<cls>'] + tokens_a + ['<sep>']， tokens是句子对而非vocab里面的tokens
  - segments：句子对信息
  - is_next：下一个句子训练样本
  - vocab[mlm_input_tokens]：15%选出的token对于语料库里面的索引
  - pred_positions：这15%token里面，被屏蔽或者被替换的索引位置（在tokens里面的索引）
  - vocab[mlm_pred_labels]：这被替换的是哪些词

在理解前面的内容后，_WikiTextDataset没什么好说的，不过是把前面的东西整合一遍罢了

In [489]:
class _WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # 输入paragraphs[i]是代表段落的句子字符串列表；
        # 而输出paragraphs[i]是代表段落的句子列表，其中每个句子都是词元列表
        paragraphs = [d2l.tokenize(
            paragraph, token='word') for paragraph in paragraphs]
        sentences = [sentence for paragraph in paragraphs
                     for sentence in paragraph]
        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>', '<mask>', '<cls>', '<sep>'])
        # 获取下一句子预测任务的数据
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(
                paragraph, paragraphs, self.vocab, max_len))
        # 获取遮蔽语言模型任务的数据
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab)
                      + (segments, is_next))
                     for tokens, segments, is_next in examples]
        # 填充输入
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(
            examples, max_len, self.vocab)
         
    # 记住这个返回的序列即可。
    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx], self.all_pred_positions[idx],
                self.all_mlm_weights[idx], self.all_mlm_labels[idx],
                self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)

接下来是吧Dataset，转变成train_iter即可

In [490]:
def load_data_wiki(batch_size, max_len):
    """加载WikiText-2数据集"""
    num_workers = d2l.get_dataloader_workers() # 4
    data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
    paragraphs = _read_wiki(data_dir)
    train_set = _WikiTextDataset(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                        shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab

In [491]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X,
     mlm_Y, nsp_y) in train_iter:
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape,
          pred_positions_X.shape, mlm_weights_X.shape, mlm_Y.shape,
          nsp_y.shape)
    break

torch.Size([512, 64]) torch.Size([512, 64]) torch.Size([512]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512])


tokens编码

In [492]:
tokens_X[:2]

tensor([[    3,     5,  2738,     2,    46,  1933,     9,     5,  1630,   137,
             2,    11,   923,    20,     5,     2,   256,   976,  8344,    21,
          1142, 16889,     8,  5576,     2,     2,   532,     4,     5,  1771,
             0,  1880,     0,    23, 13297,   619,     7,    42,     7,     5,
            75,   643,  5576,    57,  1493,    11,  3626,    86,    18,     4,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1],
        [    3,     2,     5,  2085,     6,     2,  9117,     7,    11,   412,
            14,  1544, 14825,    31,    98,    10,     0,     5,     0,     8,
          9332,     2,  2084,    14,  6591,    14,  2084,     2,  4430,     0,
            21,  1544,     4,     0,     0,  7054,    98,    16,    11,   975,
            20,   662,    14, 11487, 14143,    18,     4,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1

segment编码，表示是第几个句子

In [493]:
segments_X[:2]

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

每个句子有效长度

In [494]:
valid_lens_x[:100]

tensor([50., 47., 51., 34., 59., 53., 44., 51., 59., 64., 42., 55., 21., 41.,
        48., 58., 57., 49., 24., 46., 54., 50., 32., 48., 31., 47., 46., 53.,
        58., 24., 49., 41., 40., 38., 46., 64., 42., 47., 54., 26., 57., 60.,
        54., 23., 53., 29., 53., 41., 59., 43., 64., 44., 52., 54., 41., 43.,
        27., 64., 40., 57., 57., 43., 39., 54., 43., 43., 44., 50., 46., 52.,
        56., 50., 39., 57., 36., 45., 38., 37., 41., 42., 49., 39., 25., 53.,
        44., 46., 37., 49., 46., 40., 61., 57., 51., 37., 52., 50., 48., 37.,
        47., 43.])

待预测句子的位置编码

In [495]:
pred_positions_X[100:110]

tensor([[ 4, 10, 12, 18, 20, 32, 36, 41,  0,  0],
        [ 7, 10, 20,  0,  0,  0,  0,  0,  0,  0],
        [ 2, 12, 14, 22, 26,  0,  0,  0,  0,  0],
        [ 4,  7,  9, 30, 33, 34,  0,  0,  0,  0],
        [ 2,  9, 16, 26, 31, 33,  0,  0,  0,  0],
        [ 2,  3, 12, 14, 19, 21,  0,  0,  0,  0],
        [ 4,  5,  6, 17, 24, 30, 37, 42, 48,  0],
        [ 9, 20, 22, 26, 27, 29, 31, 35,  0,  0],
        [ 5, 13, 14,  0,  0,  0,  0,  0,  0,  0],
        [ 4,  6,  7, 10, 13, 19, 21,  0,  0,  0]])

掩码矩阵初试权重

In [496]:
mlm_weights_X[:10]

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

掩码位置

In [497]:
mlm_Y[:10]

tensor([[   13,     5,    16,   161,     6,   164,     5,    32,     0,     0],
        [    9,  6101,     7,  6591, 14825,  4430,    23,     0,     0,     0],
        [  298,    43,   169,  5899,   402,  2542, 10192,  3651,     0,     0],
        [  223,   947,    51,   157,   482,     0,     0,     0,     0,     0],
        [  197,  2678,    66,  1090,     0,  1649,   242,   466,  4434,     0],
        [   13,     0,     6,   326,    16,    11,   895,    17,     0,     0],
        [    5,  5252,     5,  3115,    12,     6,   904,     0,     0,     0],
        [   14,   124,     7,     0,   181,   102,     5,     9,     0,     0],
        [  991,   161, 10824,    20,     8,    43,    31,     0,  2196,     0],
        [ 1057,   605,  8534,   414,    28,     8,  2777,    17,     0,  2499]])